In [1]:
!pip install torch transformers sentencepiece datasets sacrebleu accelerate pyvi laonlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 207.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 76.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 170.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 183.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 97.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 117.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 118.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 120.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 195.2 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 129.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 791.7/791.7 kB 88.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 182.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 79.2 MB/s  0:00:00
  Attem

In [2]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import torch
from datasets import Dataset, concatenate_datasets
from transformers import (
    M2M100ForConditionalGeneration,
    M2M100Tokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback
)
import random

from pyvi import ViTokenizer
from laonlp.tokenize import word_tokenize as lao_word_tokenize

print(torch.cuda.get_device_name(0))
print("VRAM:", torch.cuda.get_device_properties(0).total_memory / 1e9, "GB")

[2025-12-21 14:38:52,267] INFO numexpr.utils: Note: detected 192 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-12-21 14:38:52,268] INFO numexpr.utils: Note: NumExpr detected 192 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[2025-12-21 14:38:52,268] INFO numexpr.utils: NumExpr defaulting to 8 threads.
NVIDIA H200
VRAM: 150.0217344 GB


In [3]:
def tokenize_vietnamese(text):
    """Tokenize Vietnamese text using PyVi"""
    try:
        return ViTokenizer.tokenize(text)
    except Exception as e:
        print(f"Error tokenizing Vietnamese: {e}")
        return text

def tokenize_lao(text):
    """Tokenize Lao text using LaoNLP"""
    try:
        # LaoNLP word_tokenize không có parameter engine
        tokens = lao_word_tokenize(text)
        return " ".join(tokens)
    except Exception as e:
        print(f"Error tokenizing Lao: {e}")
        return text

def tokenize_batch_vietnamese(texts):
    """Batch tokenize Vietnamese texts"""
    print(f"Tokenizing {len(texts)} Vietnamese texts...")
    results = []
    for i, text in enumerate(texts):
        results.append(tokenize_vietnamese(text))
        if (i + 1) % 10000 == 0:
            print(f"  Processed {i + 1}/{len(texts)} Vietnamese texts")
    return results

def tokenize_batch_lao(texts):
    """Batch tokenize Lao texts"""
    print(f"Tokenizing {len(texts)} Lao texts...")
    results = []
    for i, text in enumerate(texts):
        results.append(tokenize_lao(text))
        if (i + 1) % 10000 == 0:
            print(f"  Processed {i + 1}/{len(texts)} Lao texts")
    return results

print("Vietnamese and Lao tokenizers loaded successfully!")

Vietnamese and Lao tokenizers loaded successfully!


In [4]:
MODEL_NAME = "facebook/m2m100_418M"

print("Loading model and tokenizer...")
tokenizer = M2M100Tokenizer.from_pretrained(MODEL_NAME)
model = M2M100ForConditionalGeneration.from_pretrained(MODEL_NAME)
model = model.cuda()

Loading model and tokenizer...


tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

In [5]:
# ============================================================
# CELL 1: Check status
# ============================================================
# Verify
total, frozen, trainable = 0, 0, 0
for name, param in model.named_parameters():
    n = param.numel()
    total += n
    if not param.requires_grad:
        frozen += n
    else:
        trainable += n

print(f"Total params    : {total/1e6:.1f}M")
print(f"Frozen params   : {frozen/1e6:.1f}M")
print(f"Trainable params: {trainable/1e6:.1f}M")

Total params    : 483.9M
Frozen params   : 0.0M
Trainable params: 483.9M


In [6]:
DATA_DIR = "/work/data/dataset"
def load_parallel(src_file, tgt_file):
    with open(src_file, encoding="utf-8") as f:
        src = [l.strip() for l in f]
    with open(tgt_file, encoding="utf-8") as f:
        tgt = [l.strip() for l in f]
    
    assert len(src) == len(tgt)
    
    # Apply language-specific tokenization
    print("Tokenizing Vietnamese texts...")
    src_tokenized = tokenize_batch_vietnamese(src)
    
    print("Tokenizing Lao texts...")
    tgt_tokenized = tokenize_batch_lao(tgt)
    
    return Dataset.from_dict({
        "src_text": src_tokenized,
        "tgt_text": tgt_tokenized
    })


In [7]:
# Load toàn bộ dữ liệu từ train.vi và train.lo
full_dataset = load_parallel(
    f"{DATA_DIR}/train.vi",
    f"{DATA_DIR}/train.lo"
)

print(f"Total dataset size: {len(full_dataset)} examples")

# Chia dataset: 
# - Test: 1000 dòng cuối cùng
# - Dev: 3000 dòng trước test
# - Train: phần còn lại

test_size = 1000
dev_size = 3000

# Tính chỉ số
test_start_idx = len(full_dataset) - test_size
dev_start_idx = test_start_idx - dev_size

# Chia dataset
test_dataset = full_dataset.select(range(test_start_idx, len(full_dataset)))
dev_dataset = full_dataset.select(range(dev_start_idx, test_start_idx))
train_dataset = full_dataset.select(range(0, dev_start_idx))

# Shuffle training data để tránh bias thứ tự
train_dataset = train_dataset.shuffle(seed=42)

print(f"Train dataset: {len(train_dataset)} examples (for training)")
print(f"Dev dataset  : {len(dev_dataset)} examples (for validation during training)")
print(f"Test dataset : {len(test_dataset)} examples (for final evaluation)")
print("\nData split and shuffle completed.")

Tokenizing Vietnamese texts...
Tokenizing 695512 Vietnamese texts...
  Processed 10000/695512 Vietnamese texts
  Processed 20000/695512 Vietnamese texts
  Processed 30000/695512 Vietnamese texts
  Processed 40000/695512 Vietnamese texts
  Processed 50000/695512 Vietnamese texts
  Processed 60000/695512 Vietnamese texts
  Processed 70000/695512 Vietnamese texts
  Processed 80000/695512 Vietnamese texts
  Processed 90000/695512 Vietnamese texts
  Processed 100000/695512 Vietnamese texts
  Processed 110000/695512 Vietnamese texts
  Processed 120000/695512 Vietnamese texts
  Processed 130000/695512 Vietnamese texts
  Processed 140000/695512 Vietnamese texts
  Processed 150000/695512 Vietnamese texts
  Processed 160000/695512 Vietnamese texts
  Processed 170000/695512 Vietnamese texts
  Processed 180000/695512 Vietnamese texts
  Processed 190000/695512 Vietnamese texts
  Processed 200000/695512 Vietnamese texts
  Processed 210000/695512 Vietnamese texts
  Processed 220000/695512 Vietnamese 

In [8]:
MAX_LEN = 256

def preprocess(batch):
    tokenizer.src_lang = "vi"
    tokenizer.tgt_lang = "lo"

    inputs = tokenizer(
        batch["src_text"],
        truncation=True,
        max_length=MAX_LEN
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch["tgt_text"],
            truncation=True,
            max_length=MAX_LEN
        )

    inputs["labels"] = labels["input_ids"]
    return inputs


In [9]:
train_dataset = train_dataset.map(
    preprocess,
    batched=True,
    remove_columns=train_dataset.column_names,
    num_proc=8
)

dev_dataset = dev_dataset.map(
    preprocess,
    batched=True,
    remove_columns=dev_dataset.column_names,
    num_proc=8
)


Map (num_proc=8):   0%|          | 0/691512 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

Map (num_proc=8):   0%|          | 0/3000 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

In [14]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    padding=True
)


In [15]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True

In [16]:
training_args_phase1 = TrainingArguments(
    output_dir="/work/m2m_vi_lo_phase1_no_freeze",
    # Eval / Save
    eval_strategy="steps",
    eval_steps=500,
    save_steps=500,
    save_total_limit=2,

    logging_steps=100,

    # Batch (encoder frozen → đẩy lớn)
    per_device_train_batch_size=128,
    per_device_eval_batch_size=64,
    gradient_accumulation_steps=2,

    # Optim
    learning_rate=5e-5,
    lr_scheduler_type="linear",
    warmup_ratio=0.05,
    weight_decay=0.005,
    max_grad_norm=1.0,

    # Epochs
    num_train_epochs=1,

    # Precision
    bf16=True,
    fp16=False,

    # Speed
    group_by_length=True,
    dataloader_num_workers=8,
    dataloader_pin_memory=True,

    # Best model
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    report_to="none",
)


In [17]:
# ============================================================
# CELL 3: Create Trainer - Phase 1
# ============================================================
trainer_phase1 = Trainer(
    model=model,
    args=training_args_phase1,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[
        EarlyStoppingCallback(early_stopping_patience=3)
    ]
)


/tmp/ipykernel_159/1009842188.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_phase1 = Trainer(


In [18]:
# ============================================================
# CELL 4: Train Phase 1
# ============================================================
print("\n" + "="*60)
print("PHASE 1: Training no freeze ")
print("="*60 + "\n")

trainer_phase1.train()



PHASE 1: Training no freeze 



Step,Training Loss,Validation Loss
500,1.804900,1.608838
1000,1.478800,1.307796
1500,1.364900,1.198556
2000,1.284400,1.132966
2500,1.260400,1.100599


/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=2702, training_loss=1.5970751845686635, metrics={'train_runtime': 630.0011, 'train_samples_per_second': 1097.636, 'train_steps_per_second': 4.289, 'total_flos': 5.881963776304742e+16, 'train_loss': 1.5970751845686635, 'epoch': 1.0})

In [19]:
# ============================================================
# CELL 5: Save Phase 1 Model
# ============================================================
trainer_phase1.save_model("/work/m2m_vi_lo_phase1_no_freeze/best")
tokenizer.save_pretrained("/work/m2m_vi_lo_phase1_no_freeze/best")

print("\nPhase 1 model saved!")


Phase 1 model saved!


In [20]:
# ============================================================
# CELL 6: Evaluate Phase 1
# ============================================================
def translate_batch(texts, model, tokenizer, batch_size=32):
    """Batch translation for speed"""
    model.eval()
    tokenizer.src_lang = "vi"
    tokenizer.tgt_lang = "lo"
    
    outputs = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        
        inputs = tokenizer(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=256
        ).to("cuda")
        
        with torch.no_grad():
            gen = model.generate(
                **inputs,
                forced_bos_token_id=tokenizer.get_lang_id("lo"),
                num_beams=5,
                max_length=256
            )
        
        texts_out = tokenizer.batch_decode(gen, skip_special_tokens=True)
        outputs.extend(texts_out)
        
        if (i // batch_size + 1) % 10 == 0:
            print(f"Translated {i+len(batch)}/{len(texts)}")
    
    return outputs

# Lấy dữ liệu test từ test_dataset (đã chia từ train.vi/train.lo)
test_vi = test_dataset["src_text"]
test_lo = test_dataset["tgt_text"]

print(f"\nTest set size: {len(test_vi)} examples")
print("\nTranslating test set...")
preds_phase1 = translate_batch(test_vi, model, tokenizer)




Test set size: 1000 examples

Translating test set...
Translated 320/1000
Translated 640/1000
Translated 960/1000


In [21]:

from sacrebleu import corpus_bleu

# Calculate BLEU
bleu_phase1 = corpus_bleu(preds_phase1, [test_lo])
print(f"\n{'='*60}")
print(f"PHASE 1 BLEU Score: {bleu_phase1.score:.2f}")
print(f"{'='*60}\n")

# Save predictions
with open("/work/phase1_predictions.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(preds_phase1))


PHASE 1 BLEU Score: 23.28

